In [126]:
import pandas as pd
import geopandas as gpd
import requests
from bs4 import BeautifulSoup
import requests
import json
from datetime import datetime


def get_target_zips(state, city=None, zip_code=None):
    zips = pd.read_csv(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\zipcode_source\zip_code_database.csv")
    
    if city is None and zip_code is None:
        target_zips = zips[zips["state"] == state]["zip"].tolist()
    elif zip_code is None:
        target_zips = zips[(zips["primary_city"] == city) & (zips["state"] == state)]["zip"].tolist()
    else:
        target_zips = [zip_code]
    
    return target_zips


def get_stingray_rgn_id(zip):
    query_location_api = f"https://www.redfin.com/stingray/do/query-location?location={zip}&v=2"
    response = requests.get(query_location_api, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}) 
    soup = BeautifulSoup(response.text, 'html.parser').text
    prefix_removed = soup.split('&&', 1)[1]
    data = json.loads(prefix_removed)
    try:
        region_id = data["payload"]["exactMatch"].get("id").split("_",1)[1]
        return region_id
    except:
        print(f"No Exact match found for zip: {zip}")
        return None


def build_stingray_gis_params(params):
        return "&".join(f"{key}={value}" for key, value in params.items() if params.get(key) != None)




def call_stingray_rent_gis(params_url):
    api_url = "https://www.redfin.com/stingray/api/v1/search/rentals?"
    url = f"{api_url}?{params_url}"
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'})
    soup = BeautifulSoup(response.text, 'html.parser').text
    # print(url)
    data = json.loads(soup)

    return data


def parse_stingray_rent_gis(data):
    homes = data.get('homes', [])
    parsed_homes = []
    
    for home in homes:
        home_data = home.get('homeData', {})
        rental_data = home.get('rentalExtension', {})
        
        home_info = {
            "Property ID": home_data.get('propertyId'),
            "URL": home_data.get('url'),
            "Property Type": home_data.get('propertyType'),
            # "Photos Info": home_data.get('photosInfo', {}).get('photoRanges'),
            # "Static Map URL": home_data.get('staticMapUrl'),
            # "Has AT&T Fiber": home_data.get('hasAttFiber'),
            "Address": home_data.get('addressInfo', {}).get('formattedStreetLine'),
            "City": home_data.get('addressInfo', {}).get('city'),
            "State": home_data.get('addressInfo', {}).get('state'),
            "ZIP Code": home_data.get('addressInfo', {}).get('zip'),
            "Country Code": home_data.get('addressInfo', {}).get('countryCode'),
            "Latitude": home_data.get('addressInfo', {}).get('centroid', {}).get('centroid', {}).get('latitude'),
            "Longitude": home_data.get('addressInfo', {}).get('centroid', {}).get('centroid', {}).get('longitude'),
            "Rental ID": rental_data.get('rentalId'),
            "Max Beds": rental_data.get('bedRange', {}).get('max'),
            "Max Baths": rental_data.get('bathRange', {}).get('max'),
            "Max Square Feet": rental_data.get('sqftRange', {}).get('max'),
            "Max Rent Price": rental_data.get('rentPriceRange', {}).get('max'),
            # "Last Updated": rental_data.get('lastUpdated'),
            # "Number of Available Units": rental_data.get('numAvailableUnits'),
            # "Status": rental_data.get('status'),
            # "Date Available": rental_data.get('dateAvailable'),
            # "Rental Details Page Type": rental_data.get('rentalDetailsPageType'),
            # "Search Rank Score": rental_data.get('searchRankScore'),
            # "Freshness Timestamp": rental_data.get('freshnessTimestamp'),
            "Description": rental_data.get('description'),
            # "Revenue Per Lead": rental_data.get('revenuePerLead'),
            # "Feed Source Internal ID": rental_data.get('feedSourceInternalId'),
            # "Is Commercial Paid": rental_data.get('isCommercialPaid'),
            # "Feed Original Source": rental_data.get('feedOriginalSource'),
            # "Desktop Phone": rental_data.get('desktopPhone'),
            # "Mobile Web Phone": rental_data.get('mobileWebPhone'),
            # "Mobile App Phone": rental_data.get('mobileAppPhone')
        }
        parsed_homes.append(home_info)
    
    return parsed_homes

def geocode_dataframe(df, latitude_col='Latitude', longitude_col='Longitude'):
    import geopandas as gpd
    import pandas as pd
    """
    Geocode the given DataFrame based on geographic data files.

    Parameters:
    df (pd.DataFrame): DataFrame containing the data to be geocoded.
    longitude_col (str): Name of the column containing longitude values.
    latitude_col (str): Name of the column containing latitude values.

    Returns:
    pd.DataFrame: Geocoded DataFrame.
    """
    # Convert the DataFrame to a GeoDataFrame
    gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df[longitude_col], df[latitude_col]), crs="EPSG:4326"
    )

    # Load and preprocess demographic areas
    demographic_areas = gpd.read_file(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\census_block_group_source_nationwide\v107\blkgrp.gdb")
    demographic_areas.to_crs("EPSG:4326", inplace=True)
    demographic_areas["GEOID"] = (
        demographic_areas["STATE_FIPS"].astype(str).str.zfill(2)
        + demographic_areas["COUNTY_FIPS"].astype(str).str.zfill(3)
        + demographic_areas["TRACT_FIPS"].astype(str).str.zfill(6)
        + demographic_areas["BLOCKGROUP_FIPS"].astype(str)
    )
    demographic_areas = demographic_areas[["GEOID", "geometry"]].rename(columns={"GEOID": "cbg_geoid"})

    # Load and preprocess CBSA areas
    cbsa_source = gpd.read_file(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\cbsa_source\tl_2020_us_cbsa.shp")
    cbsa_source.to_crs("EPSG:4326", inplace=True)
    cbsa_source = cbsa_source[["GEOID", "NAME", "geometry"]].rename(columns={"GEOID": "cbsa_geoid", "NAME": "cbsa_name"})

    # Load and preprocess state areas
    state_source = gpd.read_file(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\state_source\States_shapefile.shp")
    state_source.to_crs("EPSG:4326", inplace=True)
    state_source = state_source[["FID", "State_Code", "geometry"]].rename(columns={"FID": "state_id", "State_Name": "state_name"})

    # Perform spatial joins
    geocoded_dots = gdf.sjoin(demographic_areas, how="left").drop(["index_right"], axis=1)
    geocoded_dots = geocoded_dots.sjoin(cbsa_source, how='left').drop(["index_right"], axis=1)
    geocoded_dots = geocoded_dots.sjoin(state_source, how='left').drop(["index_right"], axis=1)

    # Drop unnecessary columns
    geocoded_dots = geocoded_dots.drop(['geometry'], axis=1)

    return pd.DataFrame(geocoded_dots)


In [127]:
#Load Spatial Datasets
demographic_areas = gpd.read_file(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\census_block_group_source_nationwide\v107\blkgrp.gdb")
demographic_areas.to_crs("EPSG:4326", inplace=True)

demographic_areas["GEOID"] = \
    demographic_areas["STATE_FIPS"].astype(str).str.zfill(2)  \
    + demographic_areas["COUNTY_FIPS"].astype(str).str.zfill(3) \
    + demographic_areas["TRACT_FIPS"].astype(str).str.zfill(6) \
    + demographic_areas["BLOCKGROUP_FIPS"].astype(str)

demographic_areas = demographic_areas[["GEOID", "geometry"]].rename(columns={"GEOID":"cbg_geoid"})

cbsa_source = gpd.read_file(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\cbsa_source\tl_2020_us_cbsa.shp")
cbsa_source.to_crs("EPSG:4326", inplace=True)
cbsa_source = cbsa_source[["GEOID", "NAME", "geometry"]].rename(columns={"GEOID":"cbsa_geoid", "NAME": "cbsa_name"})

state_source = gpd.read_file(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\state_source\States_shapefile.shp")
state_source.to_crs("EPSG:4326", inplace=True)
state_source = state_source[["FID", "State_Code", "geometry"]].rename(columns={"FID":"state_id", "State_Name": "state_name"})

In [128]:


Zip = None
City = None
States = ["TX"]

target_zips = []

for State in States:
    target_zips.extend(get_target_zips(State, City, Zip))


print(f"Number of Zipcodes to be Scrubbed: {len(target_zips)}")

data = []

for index, zip in enumerate(target_zips):
    if index % 10 == 0:
        print(f"{index} Zip Codes Evaluated")

    params = {
        #??Active Listings
        "al": 1,
        #Rentals Only
        "isRentals":"true",
        #Include Nearby Homes
        "include_nearby_homes": "false",
        # Market. ie Seattle
        "market": None,
        # Number of homes to retrieve
        "num_homes": 350,
        #How to Sort the homes
        "ord": "days-on-redfin-asc",
        "page_number": 1,
        "poly": None,
        #Listing Types
        "sf": "1,2,3,4,5,6,7",
        "start": None,
        "status": 9,
        # User input property types (currently only single family, townhomes, multifamily : 134)
        "uipt": "1,3,4",
        # ??API Version?
        "v": 8,
        "zoomLevel": None,
        #Type of Region analyzed
        "region_type" : 2,
        "region_id" : get_stingray_rgn_id(zip)
    }

    if params.get("region_id") == None:
        continue
    else:
        url_param = build_stingray_gis_params(params)
        json_data = call_stingray_rent_gis(url_param)
        list_data = parse_stingray_rent_gis(json_data)

    data.extend(list_data)


new_rentals = pd.DataFrame(data)
new_rentals.drop_duplicates(subset=["Property ID"], inplace=True)

new_rentals["updated_date"] = datetime.now().date()


Number of Zipcodes to be Scrubbed: 2661
0 Zip Codes Evaluated
10 Zip Codes Evaluated
20 Zip Codes Evaluated
30 Zip Codes Evaluated
40 Zip Codes Evaluated
50 Zip Codes Evaluated
No Exact match found for zip: 75059
No Exact match found for zip: 75064
60 Zip Codes Evaluated
70 Zip Codes Evaluated
80 Zip Codes Evaluated
90 Zip Codes Evaluated
100 Zip Codes Evaluated
110 Zip Codes Evaluated
120 Zip Codes Evaluated
130 Zip Codes Evaluated
140 Zip Codes Evaluated
150 Zip Codes Evaluated
160 Zip Codes Evaluated
170 Zip Codes Evaluated
180 Zip Codes Evaluated
190 Zip Codes Evaluated
200 Zip Codes Evaluated
210 Zip Codes Evaluated
220 Zip Codes Evaluated
230 Zip Codes Evaluated
240 Zip Codes Evaluated
250 Zip Codes Evaluated
260 Zip Codes Evaluated
270 Zip Codes Evaluated
280 Zip Codes Evaluated
290 Zip Codes Evaluated
No Exact match found for zip: 75425
300 Zip Codes Evaluated
No Exact match found for zip: 75444
310 Zip Codes Evaluated
No Exact match found for zip: 75456
320 Zip Codes Evaluated

ReadTimeout: HTTPSConnectionPool(host='www.redfin.com', port=443): Read timed out. (read timeout=None)

In [131]:
print('Beginning Geocoding')


gdf = gpd.GeoDataFrame(
    new_rentals, geometry=gpd.points_from_xy(new_rentals["Longitude"], new_rentals["Latitude"]), crs="EPSG:4326"
)

# Perform spatial joins
geocoded_dots = gdf.sjoin(demographic_areas, how="left").drop(["index_right"], axis=1)
geocoded_dots = geocoded_dots.sjoin(cbsa_source, how='left').drop(["index_right"], axis=1)
geocoded_dots = geocoded_dots.sjoin(state_source, how='left').drop(["index_right"], axis=1)

# Drop unnecessary columns
geocoded_dots = geocoded_dots.drop(['geometry'], axis=1)

new_rentals_geocoded = pd.DataFrame(geocoded_dots)

# display(new_rentals_geocoded)


Beginning Geocoding


In [132]:
print('Beginning Deduping')

existing_rentals = pd.read_csv(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Redfin Data\rentals.csv")



existing_rentals['Property ID'] = existing_rentals['Property ID'].astype(str).str.strip()
new_rentals_geocoded['Property ID'] = new_rentals_geocoded['Property ID'].astype(str).str.strip()

# Identify common Property IDs
common_property_ids = existing_rentals[existing_rentals['Property ID'].isin(new_rentals_geocoded['Property ID'])]

# Filter out these common Property IDs from the existing rentals DataFrame
existing_rentals = existing_rentals[~existing_rentals['Property ID'].isin(common_property_ids['Property ID'])]

updated_rentals = pd.concat([existing_rentals, new_rentals_geocoded], ignore_index=True)

print(new_rentals_geocoded.shape[0], " Rentals Downloaded")
print(common_property_ids.shape[0], " Duplicate Rentals")
print(updated_rentals.shape[0], " Total Rentals in Dataset")


Beginning Deduping
28776  Rentals Downloaded
0  Duplicate Rentals
55392  Total Rentals in Dataset
